In [47]:
1+1

2

In [48]:
import os

In [49]:
%pwd

'c:\\Users\\Sarthak\\textsummarizer'

In [50]:
os.chdir("c:\\Users\\Sarthak\\textsummarizer")

In [51]:
%pwd

'c:\\Users\\Sarthak\\textsummarizer'

In [52]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file: Path
    unzip_dir: Path


In [53]:
from src.textsummarizer.constant import *
from src.textsummarizer.utils.common import read_yaml,create_directories

In [54]:
1+1

2

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumualation_steps: int

In [56]:
from src.textsummarizer.constant import *
from src.textsummarizer.utils.common import read_yaml,create_directories

In [57]:
%pwd

'c:\\Users\\Sarthak\\textsummarizer'

In [58]:
os.chdir('../')

In [59]:
import os

In [60]:
os.chdir('c:\\Users\\Sarthak\\textsummarizer')

In [61]:
%pwd

'c:\\Users\\Sarthak\\textsummarizer'

In [76]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumualation_steps: int

In [77]:
from pathlib import Path
CONFIG_FILE_PATH= Path("config/config.yaml")
PARAMS_FILE_PATH= Path("params.yaml")

class ConfigurationManager:
    def __init__(self,
                 config_path= CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumualation_steps= params.gradient_accumualation_steps,
        )

        return model_trainer_config

In [78]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk
from src.textsummarizer.entity import ModelTrainerConfig

In [79]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config=config
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator= DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

        dataset_samsum_pt=load_from_disk(self.config.data_path)
        trainer_args= TrainingArguments(
            output_dir=self.config.root_dir,num_train_epochs=1,warmup_steps=500,
            per_device_train_batch_size=1,per_device_eval_batch_size=1,
            weight_decay=0.01,logging_steps=10,
            evaluation_strategy='steps',eval_steps=500,save_steps=1e6,
            gradient_accumulation_steps=16)
        trainer = Trainer(model=model_pegasus,args=trainer_args,
                          tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"])
        trainer.train()
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        


In [72]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.49.0
Uninstalling transformers-4.49.0:
  Successfully uninstalled transformers-4.49.0
Found existing installation: accelerate 1.4.0
Uninstalling accelerate-1.4.0:
  Successfully uninstalled accelerate-1.4.0
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.4.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached accelerate-1.4.0-py3-none-any.whl (342 kB)


In [80]:
config=ConfigurationManager()
model_trainer_config=config.get_model_trainer_config()
model_trainer_config=ModelTrainer(config=model_trainer_config)
model_trainer_config.train()

[2025-02-26 15:39:28,250: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2025-02-26 15:39:28,257: INFO: common: Yaml file: params.yaml loaded successfully]
[2025-02-26 15:39:28,260: INFO: common: created directories at: artifacts]
[2025-02-26 15:39:28,262: INFO: common: created directories at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Sarthak\AppData\Local\Temp\ipykernel_18856\1353089573.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus,args=trainer_args,


Step,Training Loss,Validation Loss


c:\Users\Sarthak\textsummarizer\venv\Lib\site-packages\transformers\modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [81]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelEvaluationConfig:
    root_dir:Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [82]:
from src.textsummarizer.constant import *
from src.textsummarizer.utils.common import read_yaml,create_directories

In [88]:
class ConfigurationManager:
    def __init__(self,
                 config_path= CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evalution_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name,
        )

        return model_evaluation_config

In [89]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_from_disk
import torch
import pandas as pd

from tqdm import tqdm

In [99]:
import evaluate
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config=config
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]
    def calculate_metrics_on_test_ds(self,dataset,metrics,model,tokenizer,
                                    batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                                    column_text="article",
                                    column_summary="highlights"):
        article_batches= list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches= list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))

        for article_batch,target_batch in tqdm(
            zip(article_batches,target_batches),total=len(article_batches)):
            inputs= tokenizer(article_batch,max_length=1024,truncation=True,
                            padding="max_length",return_tensors="pt")
            summaries= model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8,num_beams=8,max_length=128)
            decoded_summaries= [tokenizer.decode(s,skip_special_tokens=True,
                                                clean_up_tokenization_spaces=True)
                                for s in summaries]
            decoded_summaries=[d.replace(""," ") for d in decoded_summaries]
            metrics.add_batch(predictions=decoded_summaries,references=target_batch)
        score= metrics.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        dataset_samsum_pt= load_from_disk(self.config.data_path)

        rouge_names= ["rouge1","rouge2","rougeL","rougeLsum"]
        rouge_metric=evaluate.load('rouge')
        #rouge_metric=rouge_metric

        score = self.calculate_metrics_on_test_ds(
            dataset_samsum_pt['test'][0:10],rouge_metric,model_pegasus,tokenizer,batch_size=2,column_text='dialogue',column_summary='summary'
        )

        rouge_dict={rn:score[rn] for rn in rouge_names}
        df=pd.DataFrame(rouge_dict,index=['pegasus'])
        df.to_csv(self.config.metric_file_name,index=False)

In [100]:
config=ConfigurationManager()
model_evaluation_config=config.get_model_evalution_config()
model_evaluation_config=ModelEvaluation(config=model_evaluation_config)
model_evaluation_config.evaluate()

[2025-02-26 16:52:33,608: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2025-02-26 16:52:33,611: INFO: common: Yaml file: params.yaml loaded successfully]
[2025-02-26 16:52:33,613: INFO: common: created directories at: artifacts]
[2025-02-26 16:52:33,614: INFO: common: created directories at: artifacts/model_evaluation]


100%|██████████| 5/5 [04:59<00:00, 59.85s/it]

[2025-02-26 16:57:36,681: INFO: rouge_scorer: Using default tokenizer.]
